In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
import NuestrasFunciones

from names_dataset import NameDataset
nd = NameDataset()
from countryinfo import CountryInfo


from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer 
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import FunctionTransformer

In [2]:
df_titanic = pd.read_csv("In/train.csv")

In [54]:
df_titanic.Age[5]

nan

In [3]:
def limpia(df):
    df.rename(columns = {"SibSp": "SiblingsSpouses","Parch":"ParentsChildren"},inplace = True)
    #df.drop("PassengerId",axis = 1,inplace = True)

    df.loc[df["Sex"] == "male", "Sex"] = 1
    df.loc[df["Sex"] == "female", "Sex"] = 0
    df["Sex"] =  df["Sex"].astype("category")

        
    return df       

In [4]:
def cant_mismo(tick,ser):
    lista = []
    val_counts = ser.value_counts()
    valor = val_counts[tick] -1
    return valor


In [5]:
def Operaciones_Ticekt(df):
    x = df['Ticket'].apply(lambda x: x.split(" "))
    
    df["PostTicket"] = [i[1] if len(i) == 2 else i[0] for i in x]
    df.Ticket = df.Ticket.apply(lambda x: x.replace(",","").replace(".","").replace("/","").upper())
    df["Cant_PersonasMismoTicket"] = df.Ticket.apply(lambda x: cant_mismo(x,df.Ticket))


    df["PreTicket"] = [i[0] if len(i) == 2 else "desconocido" for i in x]
    df.PreTicket = df.PreTicket.apply(lambda x: x.replace(",","").replace(".","").replace("/","").upper())
    df.PreTicket = df.PreTicket.apply(lambda x: x[0] if x[0].lower() != "s" else x[0:2])
    #df_titanic.PreTicket = df_titanic.PreTicket.apply(lambda x: x if x in ["SC","SO"] else x[0])
    df.PreTicket = df.PreTicket.apply(lambda x: x if x == "SO" else x[0])
    df.PreTicket = df.PreTicket.apply(lambda x: "DESC" if x == "D" else x)
    
    return df


In [6]:
def disgrega_titulo(df):
    df = df.copy()
    df["Titulo_Expandido"] = df["Titulo"]
    for i in df.Titulo_Expandido:
        #print(i)
        if i in["Mr","Mrs","Miss","Master"]:
            
            df.loc[((df["ParentsChildren"] == 0) & (df["Titulo_Expandido"] == i)),"Titulo_Expandido"] = f"{i}_SinParentsChildren"
            df.loc[((df["ParentsChildren"] != 0) & (df["Titulo_Expandido"] == i)),"Titulo_Expandido"] = f"{i}_ConParentsChildren"
        
        
            df.loc[((df["Cantidad_MismoApellido"] == 0) & (df["Titulo_Expandido"] == f"{i}_SinParentsChildren")),"Titulo_Expandido"] = f"{i}_SinParentsChildren_SinPersMismoApe"
            df.loc[((df["Cantidad_MismoApellido"] != 0) & (df["Titulo_Expandido"] == f"{i}_SinParentsChildren")),"Titulo_Expandido"] = f"{i}_SinParentsChildren_ConPersMismoApe"
            df.loc[((df["Cantidad_MismoApellido"] == 0) & (df["Titulo_Expandido"] == f"{i}_ConParentsChildren")),"Titulo_Expandido"] = f"{i}_ConParentsChildren_SinPersMismoApe"
            df.loc[((df["Cantidad_MismoApellido"] != 0) & (df["Titulo_Expandido"] == f"{i}_ConParentsChildren")),"Titulo_Expandido"] = f"{i}_ConParentsChildren_ConPersMismoApe"
        
        
            df.loc[((df["SiblingsSpouses"] != 0) & (df["Titulo_Expandido"] == f"{i}_SinParentsChildren_SinPersMismoApe")),"Titulo_Expandido"] = f"{i}_ConSoloHermanastros"
    return df


In [7]:
def return_pais(ape):

    try:
        dict = nd.search(ape)["last_name"]["country"]
        maximo = max(zip(dict.values(),dict.keys()))[1]
    except TypeError:
        try:
           dict = nd.search(ape)["first_name"]["country"]
           maximo = max(zip(dict.values(),dict.keys()))[1]
        except TypeError:
            maximo = "Desconodido"
    return maximo

def get_language(pais):
    try:
        idioma = CountryInfo(pais).languages()[0]
    except:
        return "Desc"
    return idioma 


In [8]:
def Operaciones_Nombres(df):
    
    x = df.Name.apply(lambda x: x.split(","))
    df["Apellido"] = df.Name.apply(lambda x: x.split(",")[0])
    df["Cantidad_MismoApellido"] = df["Apellido"].apply(lambda x: cant_mismo(x,df.Apellido))

    df["Titulo"] = df.Name.apply(lambda x: x.split(",")[1].split(".")[0].replace(" ",""))

    df = disgrega_titulo(df)

    df["Pais"] = df.Apellido.apply(lambda x: return_pais(x))  
    df["Idioma"] = df.Pais.apply(lambda x: get_language(x))
    return df



In [9]:
def feat_eng(df):
    df = Operaciones_Ticekt(df)
    df = Operaciones_Nombres(df)        
    return df

#### El test también tiene valores missing

# Te falta conceptualizar los imputadores y arreglar bugs

In [10]:
def Imputa_cabin(df):
    #Fit
    df_ApeCabin = df[["Apellido","Cabin"]].copy().dropna()#.reset_index(drop = True)
    df_TicketCabin = df[["PostTicket","Cabin"]].copy().dropna()#.reset_index(drop = True)
    
    Apellidos_ConCabin = list(df_ApeCabin.Apellido.values)
    Tickets_ConCabin = list(df_TicketCabin.PostTicket.values)
    #Transform
    for idx,(ape, cabina,tick) in enumerate(zip(df.Apellido,df.Cabin,df.PostTicket)):
        
        if cabina is np.nan:
            if ape in Apellidos_ConCabin:
                #print(ape)
                df.loc[idx, "Cabin"] = df_ApeCabin.loc[df["Apellido"] == ape, "Cabin"].values[0]
            elif tick in Tickets_ConCabin:
                df.loc[idx, "Cabin"] = df_TicketCabin.loc[df["PostTicket"] == tick, "Cabin"].values[0]
            else:
                df.loc[idx, "Cabin"] = "desconocido"

    df["Cabin_Letra"] = df.Cabin.apply(lambda x: x[0] if x != "desconocido" else "desc")
    return df

def imputa_age(df, titulo_expanido = False):

    if titulo_expanido:
        #Fit
        llave = df[["Age","Titulo_Expandido"]]
        llave = llave.groupby("Titulo_Expandido").agg("mean").reset_index()
        #Transform
        for idx, val in enumerate(df.Titulo_Expandido):
            if np.isnan(df.Age[idx]):
                df.loc[idx, "Age"] = llave.loc[llave["Titulo_Expandido"] == val, "Age"].values[0]
    
    else:
        df_agg = df[["Age","Titulo"]]
        llave = df_agg.groupby("Titulo").agg("mean").reset_index()
        for idx, val in enumerate(df.Titulo):
            if np.isnan(df.Age[idx]):
                df.loc[idx, "Age"] = llave.loc[llave["Titulo"] == val, "Age"].values[0]
    return df

### Aplicando lo aprendido en hands on machine learning de Aurelién Gerón, al imputar el test se hará con los datos del train.

In [78]:
class Imputador(BaseEstimator, TransformerMixin):

    def __init__(self, titulo_expandido = False):
        self.titulo_expandido = titulo_expandido  
        
    def fit(self, X, y=None):
        #CABIN
        
        self.df_ApeCabin = X[["Apellido","Cabin"]].dropna().copy()#.reset_index(drop = True)
        self.df_TicketCabin = X[["PostTicket","Cabin"]].dropna().copy()#.reset_index(drop = True)
        self.Apellidos_ConCabin = list(self.df_ApeCabin.Apellido.values)
        self.Tickets_ConCabin = list(self.df_TicketCabin.PostTicket.values)
        
        #AGE
        if self.titulo_expandido:
            self.llave_titulo = X[["Age","Titulo_Expandido"]]
            self.llave_titulo = self.llave_titulo.groupby("Titulo_Expandido").agg("mean").reset_index()
        else: 
            self.llave_titulo = X[["Age","Titulo"]]
            self.llave_titulo = self.llave_titulo.groupby("Titulo").agg("mean").reset_index()
    
        return self 
    
    def transform(self, X, y=None): 
        #CABIN
        for idx,(ape, cabina,tick) in enumerate(zip(X.Apellido,X.Cabin,X.PostTicket)):
            if cabina is np.nan:
                if ape in self.Apellidos_ConCabin:
                    #print(ape)
                    X.loc[idx, "Cabin"] = self.df_ApeCabin.loc[X["Apellido"] == ape, "Cabin"].values[0]
                    
                elif tick in self.Tickets_ConCabin:
                    X.loc[idx, "Cabin"] = self.df_TicketCabin.loc[X["PostTicket"] == tick, "Cabin"].values[0]
                    
                else:
                    X.loc[idx, "Cabin"] = "desconocido"
    
        X["Cabin_Letra"] = X.Cabin.apply(lambda x: x[0] if x != "desconocido" else "desc")

        #AGE
        for idx, (val, val_exp) in enumerate(zip(X.Titulo, X.Titulo_Expandido)):
            #print("-------------------------------------------------")
            #print(self.llave_titulo)
            #print("el error?")
            #print(X.Age)
            
            if self.titulo_expandido:
                if np.isnan(X.Age[idx]):
                    X.loc[idx, "Age"] = self.llave_titulo.loc[self.llave_titulo["Titulo_Expandido"] == val_exp, "Age"].values[0]
        
            else:
                if np.isnan(X.Age[idx]):
                    X.loc[idx, "Age"] = self.llave_titulo.loc[self.llave_titulo["Titulo"] == val, "Age"].values[0]

        return X



        

In [73]:
def rank_columna_fit(df,columna):
    #fit
    # Creo title ranks del df antes de sub-categorizar los títulos por si alguna de las subcategorías no aparece en el test
    df_agg = df[[columna,"Survived"]].copy()
    df_agg = df_agg.groupby([columna], as_index = False).agg("mean").sort_values(by = "Survived")

    return df_agg

In [74]:
def rank_columna_transform(df, columna, df_agg):
    arr_rank = []
    for i in df[columna]:
        
        if i in list(df_agg[columna]):
            arr_rank.append(df_agg.loc[df_agg[columna] == i, "Survived"].values[0])

        else:
            arr_rank.append(0.383838)

    #Cambio el valor numérico por rankings del 1 al 19 para que el valor de la variable no sea combinación lineal directa de Y.
    # Algoritmo que agarra el df sorted by Titulo_Rank y le asigna una categoría cardinal a cada score.
    columna += "_Rank"
    df[columna] = pd.Series(arr_rank)
    df = df.sort_values(by = columna, inplace = False).reset_index(drop = True)
    
    arr_intermed = list(df[columna])
    #Todos los scores iguales a w tendrán el mismo ranking, cuando el score no sea igual a w, se actualiza el ranking a asignar.
    w = arr_intermed[0]
    j = 1
    for i in range(len(df[columna])):
        if  arr_intermed[i] == w:
            arr_intermed[i] = j
        else:
            w = arr_intermed[i]
            j += 1
            arr_intermed[i] = j

    df[columna] = pd.Series(arr_intermed)
        
    return df    

In [75]:
class asigna_ranks(BaseEstimator, TransformerMixin):

    def __init__(self, usa_titulo_expandido = True):
        self.usa_titulo_expandido = usa_titulo_expandido

    def fit(self, X, y=None):
        self.PreTicket_Rank = rank_columna_fit(X,"PreTicket")
        self.Title_Expandido_Rank = rank_columna_fit(X,"Titulo_Expandido")
        self.Title_Rank = rank_columna_fit(X,"Titulo")
        #self.Cabin_Letra_Rank = rank_columna_fit(X,"Cabin_Letra")
        self.Idioma_Rank = rank_columna_fit(X,"Idioma")

        return self 
        
    def transform(self,X,y = None):
        X = rank_columna_transform(X,"PreTicket",self.PreTicket_Rank)
        X = rank_columna_transform(X,"Titulo",self.Title_Rank)
        X = rank_columna_transform(X,"Titulo_Expandido",self.Title_Expandido_Rank)
        #X = rank_columna_transform(X,"Cabin_Letra",self.Cabin_Letra_Rank)
        X = rank_columna_transform(X,"Idioma",self.Idioma_Rank)

        return X

In [81]:
pipep = Pipeline([
            ('Limpiar', FunctionTransformer(limpia)),
            ('feat_eng', FunctionTransformer(feat_eng)),
            ('imputa', Imputador()),
            ('Rank_assign', asigna_ranks()),
            
        ])
resultado = pipep.fit_transform(df_titanic.copy())
#df_titanic

In [82]:
resultado

,PassengerId,Survived,Pclass,Name,Sex,Age,SiblingsSpouses,ParentsChildren,Ticket,Fare,...,Cantidad_MismoApellido,Titulo,Titulo_Expandido,Pais,Idioma,Cabin_Letra,PreTicket_Rank,Titulo_Rank,Titulo_Expandido_Rank,Idioma_Rank
0,653,0,3,"Kalvik, Mr. Johannes Halvorsen",1,21.000000,0,0,8475,8.4333,...,0,Mr,Mr_SinParentsChildren_SinPersMismoApe,Norway,no,desc,5,2,4,1
1,700,0,3,"Humblen, Mr. Adolf Mathias Nicolai Olsen",1,42.000000,0,0,348121,7.6500,...,0,Mr,Mr_SinParentsChildren_SinPersMismoApe,Norway,no,F,5,2,4,1
2,409,0,3,"Birkeland, Mr. Hans Martin Monsen",1,21.000000,0,0,312992,7.7750,...,0,Mr,Mr_SinParentsChildren_SinPersMismoApe,Norway,no,desc,5,2,4,1
3,15,0,3,"Vestrom, Miss. Hulda Amanda Adolfina",0,14.000000,0,0,350406,7.8542,...,0,Miss,Miss_SinParentsChildren_SinPersMismoApe,Norway,no,desc,5,6,14,1
4,520,0,3,"Pavlovic, Mr. Stefo",1,32.000000,0,0,349242,7.8958,...,0,Mr,Mr_SinParentsChildren_SinPersMismoApe,Serbia,rs,desc,5,2,4,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,850,1,1,"Goldenberg, Mrs. Samuel L (Edwiga Grabowska)",0,35.898148,1,0,17453,89.1042,...,1,Mrs,Mrs_SinParentsChildren_ConPersMismoApe,Israel,he,C,5,7,16,14
887,293,0,2,"Levy, Mr. Rene Jacques",1,36.000000,0,0,SCPARIS 2163,12.8750,...,0,Mr,Mr_SinParentsChildren_SinPersMismoApe,Israel,he,D,6,2,4,14
888,205,1,3,"Cohen, Mr. Gurshon ""Gus""",1,18.000000,0,0,A5 3540,8.0500,...,0,Mr,Mr_SinParentsChildren_SinPersMismoApe,Israel,he,desc,1,2,4,14
889,760,1,1,"Rothes, the Countess. of (Lucy Noel Martha Dye...",0,33.000000,0,0,110152,86.5000,...,0,theCountess,theCountess,Portugal,pt,B,5,8,19,14
